## 한국은행 기준금리

In [1]:
import re
import requests
import matplotlib.pyplot as plt
%matplotlib inline
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

In [10]:
url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643'
resp = requests.get(url)
soup = BeautifulSoup(resp.content,'lxml')
td_tag = soup.select('div.table.tac tr td')
context = [td.text for td in td_tag]

dic_baserate = {}
dic_baserate['변경년도'],dic_baserate['변경월일'],dic_baserate['기준금리'] = [], [] ,[]

for i in range(int(len(context)/3)):
  dic_baserate['변경년도'].append(context[i*3])
  dic_baserate['변경월일'].append(context[i*3+1])
  dic_baserate['기준금리'].append(float(context[i*3+2]))

dic_baserate['날짜'] = []
for i in range(len(dic_baserate['변경월일'])):
  dic_baserate['날짜'].append(dic_baserate['변경년도'][i]+'-'+dic_baserate['변경월일'][i].replace('월','').replace('일','').replace(' ','-'))

base_rate = pd.DataFrame(dic_baserate)
cols = list(base_rate)
base_rate = base_rate.loc[5:33][cols[2:]] #2004년 11월부터 2017년 12월까지

In [77]:
dic_date = {}
dic_date['날짜'] = pd.date_range("2004-11-11","2017-12-31").astype(str)
date = pd.DataFrame(dic_date)

data = date.merge(base_rate,how='outer')
for i in range(len(data)-1):
    if pd.isna(data.iloc[i,1]) == False and pd.isna(data.iloc[i+1,1]) == True:
        data.iloc[i+1,1] = data.iloc[i,1]

data1 = data.iloc[171:] #2005년 5월부터
data2 = data.iloc[141:-30] #2005년 4월부터
data1 = data1.reset_index(drop=True)
data2 = data2.reset_index(drop=True)

In [78]:
df_baserate = pd.concat([data1, data2],axis=1)

In [79]:
df_baserate['label']= df_baserate.iloc[:,1]- df_baserate.iloc[:,3]

In [80]:
import numpy as np

In [81]:
label = []
for i in range(len(df_baserate)):
    if df_baserate['label'][i] == 0:
        label.append('same')
    elif df_baserate['label'][i] > 0:
        label.append('up')
    else:
         label.append('down')

In [82]:
df_baserate['labeling'] = label

,날짜,기준금리,날짜,기준금리,label,labeling
0,2005-05-01,3.25,2005-04-01,3.25,0.0,same
1,2005-05-02,3.25,2005-04-02,3.25,0.0,same
2,2005-05-03,3.25,2005-04-03,3.25,0.0,same
3,2005-05-04,3.25,2005-04-04,3.25,0.0,same
4,2005-05-05,3.25,2005-04-05,3.25,0.0,same


In [83]:
df_baserate.to_csv('df_base_rate',index=False)